In [1]:
import sys

sys.path.append("../")

import numpy as np
import pandas_ta as ta
import joblib

from terminal.Extensions.MT5Ext import MT5Ext

from sklearn.metrics import accuracy_score

In [2]:
# Carregar modelos
bought_model = joblib.load("../models/bought.joblib")
sold_model = joblib.load("../models/sold.joblib")

In [3]:
def feature_target(file):
    # Carregar os dados
    ticks = np.load(file, allow_pickle=True)["ticks"]
    data = MT5Ext.create_ohlc_from_ticks(ticks, "2s")

    del ticks

    # Cálculo de indicadores
    data["EMA_1"] = data.ta.ema(3)
    data["EMA_2"] = data.ta.ema(8)
    data["EMA_3"] = data.ta.ema(16)

    data["EMA_1_CHANGE"] = data["EMA_1"].pct_change()
    data["EMA_2_CHANGE"] = data["EMA_2"].pct_change()
    data["EMA_3_CHANGE"] = data["EMA_3"].pct_change()

    data.dropna(inplace=True)

    # Pré-processamento de dados
    data["CURRENT_CHANGE"] = data["close"].pct_change()
    data["FORECAST_CHANGE"] = data["CURRENT_CHANGE"].shift(-1)

    data["TARGET"] = np.where(
        (data["CURRENT_CHANGE"] > 0) & (data["FORECAST_CHANGE"] > 0)
        | (data["CURRENT_CHANGE"] < 0) & (data["FORECAST_CHANGE"] < 0),
        1,
        0,
    )

    data.dropna(inplace=True)

    bought_data = data[data["close"] > data["open"]]
    sold_data = data[data["close"] < data["open"]]

    # Criar características e rótulos
    feature_columns = [
        "CURRENT_CHANGE",
        "EMA_1_CHANGE",
        "EMA_2_CHANGE",
        "EMA_3_CHANGE"
    ]

    bought_data = data[data["close"] > data["open"]]
    sold_data = data[data["close"] < data["open"]]

    bought_features = bought_data[feature_columns]
    bought_target = bought_data["TARGET"]

    sold_features = sold_data[feature_columns]
    sold_target = sold_data["TARGET"]

    del data
    del bought_data
    del sold_data

    return bought_features, bought_target, sold_features, sold_target


def test(file):
    bought_features, bought_target, sold_features, sold_target = feature_target(file)

    # Avaliar o modelo
    y_pred = bought_model.predict(bought_features)
    accuracy = accuracy_score(bought_target, y_pred)
    print(f"bought accuracy: {accuracy:.2f}")
    
    # Avaliar o modelo
    y_pred = sold_model.predict(sold_features)
    accuracy = accuracy_score(sold_target, y_pred)
    print(f"sold accuracy: {accuracy:.2f}")

In [4]:
test("./data/ticks_WIN$N_2024_7.npz")

bought accuracy: 0.77
sold accuracy: 0.77
